### **What is Pydantic?**

**Pydantic** is a Python library for **data validation and settings management** using Python type annotations. It enforces type hints at runtime and provides powerful tools for parsing and validating structured data like JSON.
It’s widely used in modern Python frameworks like **FastAPI**, where request/response data needs to be validated.

---

### **Why is Pydantic Used?**

* ✅ To validate input data automatically.
* ✅ To parse data from external sources (like JSON APIs).
* ✅ To provide clear and explicit error messages when data validation fails.
* ✅ To convert compatible data types automatically where possible (e.g., a string `"23"` to an integer `23`).
* ✅ To build robust, type-safe, and self-documenting code.

---

### **Basic Example: Defining a Pydantic Model**

Let’s define a **Patient** model with the following fields:

* `pid: int`
* `name: str`
* `allergies: list[str]`
* `contact_no: int`


In [1]:
from pydantic import BaseModel

class Patient(BaseModel):
    pid: int
    name: str
    allergies: list[str]
    contact_no: int

### **Creating an Object and Input Format**

Pydantic models typically accept input data in the form of a **dictionary (Python’s native data type)**, often deserialized from **JSON**.

✅ **Example Input (as a Python dict / JSON)**

In [2]:
patient_data = {
    "pid": 101,
    "name": "Ankush",
    "allergies": ["Dust", "Peanuts"],
    "contact_no": 9876543210
}

patient = Patient(**patient_data)
print(patient)

pid=101 name='Ankush' allergies=['Dust', 'Peanuts'] contact_no=9876543210


### **Type Coercion (Automatic Type Conversion)**

Pydantic tries to **coerce data types** if possible.
For example, if `pid` is defined as an `int`, and you pass `"23"` (a string), it will convert it to `23`.
However, if conversion is impossible (like converting `"Ankush"` to `int`), it raises a `ValidationError`.

✅ **Example: Auto Conversion**

In [3]:
patient_data = {
    "pid": "23",  # string, but should be int
    "name": "Ankush",
    "allergies": ["Dust"],
    "contact_no": "9876543210"  # string, but should be int
}

patient = Patient(**patient_data)
print(patient)


pid=23 name='Ankush' allergies=['Dust'] contact_no=9876543210


### **Exception Handling When Type Conversion Fails**

If type coercion isn’t possible (like converting `"Ankush"` to `int`), Pydantic will raise a `ValidationError`.

❌ **Example: Type Mismatch Error**

In [4]:
from pydantic import ValidationError

invalid_data = {
    "pid": "Ankush",  # invalid int
    "name": "Ankush",
    "allergies": ["Dust"],
    "contact_no": "9876543210"
}

try:
    patient = Patient(**invalid_data)
except ValidationError as e:
    print(e)

1 validation error for Patient
pid
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Ankush', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


### **Required vs Optional Fields in Pydantic**

By default, **all fields in a Pydantic model are required**.
If any field is missing in the input data, Pydantic will raise a `ValidationError`.

✅ **Example: All Fields Required**



In [5]:
from pydantic import BaseModel, ValidationError

class Patient(BaseModel):
    pid: int
    name: str
    allergies: list[str]
    contact_no: int

# Missing contact_no
data = {
    "pid": 23,
    "name": "Ankush",
    "allergies": ["Dust"]
}

try:
    patient = Patient(**data)
except ValidationError as e:
    print(e)

1 validation error for Patient
contact_no
  Field required [type=missing, input_value={'pid': 23, 'name': 'Anku..., 'allergies': ['Dust']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing


### **Making a Field Optional**

To make a field optional, we can use `Optional` from the `typing` module.
An optional field can either have a value of the specified type or be `None`.

We can also provide **any default value** to a field, which makes it optional because if the value is missing, the default will be used.

✅ **Example: Optional Field**

In [6]:
from typing import Optional

class Patient(BaseModel):
    pid: int
    name: str
    allergies: list[str]
    contact_no: Optional[int] = None  # Optional with a default value

### **Built-in Data Type Validators in Pydantic**

Pydantic provides many pre-created custom data type validators.
Some common ones include:

* `EmailStr` → Validates a valid email address.
* `AnyUrl` → Validates a valid URL.

✅ **Example: Using EmailStr**

In [7]:
from pydantic import BaseModel, EmailStr

class User(BaseModel):
    email: EmailStr

user = User(email="ankush@example.com")
print(user)

email='ankush@example.com'


❌ If an invalid email is passed, it will raise a `ValidationError`.

### **Using `Field()` for Extra Validation and Metadata**

The `Field()` function is used to provide:

* Default values
* Metadata (like title, description)
* Constraints (like `gt=0`, `max_length=100`)

✅ **Example: Adding Constraints with Field()**

In [8]:
from pydantic import BaseModel, Field
from typing import Optional

class Product(BaseModel):
    id: int
    name: str = Field(..., max_length=50, title="Product Name")
    price: Optional[float] = Field(default=None, gt=0, description="Price must be greater than zero")

product1 = Product(id=1, name="Oven")
print(product1)


id=1 name='Oven' price=None



* `...` means the field is **required**
* `gt=0` means value must be **greater than zero**
* `max_length` sets maximum length for strings
* `title` and `description` add metadata

### **Disabling Type Coercion with `strict=True`**

By default, Pydantic will try to **coerce data types**.
If you want to enforce strict type checking and prevent coercion, you can use the `strict=True` parameter inside `Field()`.

✅ **Example: Enforcing Strict Types**

In [9]:
from pydantic import BaseModel, Field, ValidationError

class Item(BaseModel):
    quantity: int = Field(..., strict=True)

# This will raise a ValidationError since "10" is a string, not int
try:
    item = Item(quantity="10")
except ValidationError as e:
    print(e)


1 validation error for Item
quantity
  Input should be a valid integer [type=int_type, input_value='10', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_type


### **What is a Field Validator?**

In Pydantic, **field validators** are methods used to perform **custom, complex data validation** logic on specific fields.
They’re especially useful when default type validation isn't enough — for example, checking email domains, transforming input values, or enforcing custom constraints.


### **Defining a Field Validator**

Use the `@field_validator` decorator to associate a method with a specific field.
This method should be a `classmethod` and must return the validated (or transformed) value.

✅ **Basic Syntax:**

```python
@field_validator('field_name')
@classmethod
def method_name(cls, value):
    # validation logic
    return value
```

### **Modes in Field Validators: `"before"` and `"after"`**

* **mode="after" (default):**

  * Type coercion happens first.
  * The validator runs after the type has been converted.

* **mode="before":**

  * Validator runs **before type coercion**.
  * The raw input is passed directly.
  * Useful when you need to validate or transform raw input before type casting.

### **Example: Custom Field Validators in Action**

In [10]:
from pydantic import BaseModel, field_validator, ValidationError

class User(BaseModel):
    name: str
    age: int 
    email: str

    # ✅ Email Validator
    @field_validator('email')
    @classmethod
    def validate_email(cls, value):
        valid_email_domains = ['okhdfc.com', 'icicibank.com']
        extension = value.split('@')[-1]
        if extension not in valid_email_domains:
            raise ValueError(f"Email must end with one of {valid_email_domains}")
        return value

    # ✅ Name Transformation to Uppercase
    @field_validator('name')
    @classmethod
    def upper_case(cls, value):
        return value.upper()

    # ✅ Age Validation with mode="before"
    @field_validator('age', mode='before')
    @classmethod
    def validate_age(cls, value):
        try:
            if 0 < value < 100:
                return value
            else:
                raise ValueError("Pass the age within valid range [0-100]")
        except TypeError:
            print("Comparison should be performed between integers")


try:
    person1 = User(name="Ankush", age=24, email="ankush@okhdfc.com")
    print(f"Person1 Output : {person1}\n")

    person2 = User(name="Ankush", age="24", email="ankush@okhdfc.com")
    print(f"Person2 Output : {person2}\n")

    # This will raise a validation error
    person3 = User(name="Ankush", age=150, email="ankush@gmail.com")

except ValidationError as e:
    print(e)


Person1 Output : name='ANKUSH' age=24 email='ankush@okhdfc.com'

Comparison should be performed between integers
1 validation error for User
age
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/int_type


### **What is a Model Validator?**

While `@field_validator` validates **individual fields**, a **model validator** is used when you need to:

* Validate logic that depends on **multiple fields together**
* Perform consistency checks across different fields
* Transform or adjust the entire model object after creation

You define a model validator using the `@model_validator` decorator.SS

### **Summary**

| Field Validator              | Model Validator                         |
| :--------------------------- | :-------------------------------------- |
| Validates a **single field** | Validates **multiple fields together**  |
| Uses `@field_validator`      | Uses `@model_validator`                 |
| Receives individual value    | Receives the entire model object        |
| Can modify or validate value | Can validate or modify the entire model |


### **Key Points**

* Use `@model_validator` when validation logic involves **multiple fields together**.
* Use `mode='after'` for working with converted and validated field values.
* Raise `ValueError` with clear messages for failed validations.
* Always **return the model** instance in `mode='after'` validators.


### **Example: Multi-Field Validation with `@model_validator`**

In [11]:
from pydantic import BaseModel, model_validator, ValidationError

class User(BaseModel):
    name: str
    age: int
    email: str

    @model_validator(mode='after')
    @classmethod
    def validate(cls, model):
        # ✅ Age Check
        if not (0 < model.age < 100):
            raise ValueError("The age is not in a valid range (1-99).")

        # ✅ Email Domain Check
        valid_domains = ['okhdfc', 'icicibank']
        domain = model.email.split("@")[-1]
        if domain not in valid_domains:
            raise ValueError(f"Email must end with one of {valid_domains}")

        return model


try:
    person1 = User(name="Ankush", age=25, email="ankush@icicibank")
    print(person1)
except ValidationError as e:
    print(e)

name='Ankush' age=25 email='ankush@icicibank'




### **What is a Computed Field?**

A **computed field** is a value that:

* **Is not stored** in the model's internal data dictionary
* Is **computed dynamically** based on other model fields when accessed
* Can optionally be included in the model's `__repr__`, `dict()`, and `json()` outputs

In Pydantic, you can create computed fields using the `@computed_field` decorator combined with a `@property`.


### **How It Works**

* `@property` turns the method into an instance attribute.
* `@computed_field` tells Pydantic this is a computed property.
* It’s computed on the fly whenever accessed.
* Not stored inside the model's data (i.e., not part of `__dict__`).


### **Why Use Computed Fields?**

✅ To dynamically compute values derived from other fields
✅ To avoid redundant storage of derived values
✅ To keep data consistent, as the computed value always reflects current field values
✅ Useful for things like:

* BMI calculations
* Full names from first and last name
* Tax or discount calculations

### **Example: Calculating BMI with a Computed Field**

In [12]:
from pydantic import BaseModel, computed_field

class User(BaseModel):
    name: str
    age: int
    height: float  # in meters
    weight: float  # in kg

    @computed_field
    @property
    def bmi(self) -> float:
        return round(self.weight / (self.height ** 2), 2)


try:
    person1 = User(name="Ankush", age=25, height=1.75, weight=70)
    
    print(person1)
    # ✅ __repr__ includes computed fields like bmi
    
    print(person1.bmi)
    # ✅ Computes BMI dynamically based on height and weight
    
    print(person1.__dict__)
    # ✅ No 'bmi' stored — it's computed when accessed

except Exception as e:
    print("Some error occurred:", e)

name='Ankush' age=25 height=1.75 weight=70.0 bmi=22.86
22.86
{'name': 'Ankush', 'age': 25, 'height': 1.75, 'weight': 70.0}


### **What is a Nested Model?**

A **nested model** in Pydantic is when one model class is used as a **field inside another model**.
This allows you to represent **hierarchical or structured data** cleanly and enforce validation recursively at each level.


### **Why Use Nested Models?**

* To logically group related data together (like an Address inside a Patient profile)
* To ensure validation and type checking at **each nested level**
* To simplify complex data structures into manageable, reusable model components
* To support nested JSON-like structures naturally

### **Example: Patient and Address as Nested Models**

In [13]:
from pydantic import BaseModel

class Address(BaseModel):
    city : str
    state : str 
    pin : int 

class Patient(BaseModel):
    name : str 
    gender : str 
    age : int 
    address : Address


address_dict = {"city":"Jamner","state":"Maharashtra","pin":424206}

address = Address(**address_dict)

patient_dcit = {"name":"Ankush","gender":"Male","age":26,"address":address}

patient = Patient(**patient_dcit)

print(patient)
print(patient.address)
print(patient.address.state)

name='Ankush' gender='Male' age=26 address=Address(city='Jamner', state='Maharashtra', pin=424206)
city='Jamner' state='Maharashtra' pin=424206
Maharashtra


In [14]:
nested_patient_dict = {
    "name": "Piyush",
    "gender": "Male",
    "age": 26,
    "address": {
        "city": "Jalgaon",
        "state": "Maharashtra",
        "pin": 424206
    }
}

patient = Patient(**nested_patient_dict)
print(patient)

name='Piyush' gender='Male' age=26 address=Address(city='Jalgaon', state='Maharashtra', pin=424206)


### **What is Serialization?**

**Serialization** is the process of converting a Pydantic model into a native **Python dict** or **JSON string** so it can be stored, transmitted, or displayed.

Pydantic models offer methods for this:

* `model.model_dump()` → returns a Python dictionary
* `model.model_dump_json()` → returns a JSON string

Both support powerful options like `include`, `exclude`, `exclude_unset`, and more.


### **Serialization Methods**

| Method         | Description                             |
| :------------- | :-------------------------------------- |
| `model.model_dump()` | Converts the model into a Python `dict` |
| `model.model_dump_json()` | Converts the model into a JSON `str`    |



### **Serialization Options**

| Option             | Description                                                           |
| :----------------- | :-------------------------------------------------------------------- |
| `include`          | List or set of fields to include in the output                        |
| `exclude`          | List or set of fields to exclude from the output                      |
| `exclude_unset`    | If `True`, exclude fields that weren’t explicitly set (used defaults) |
| `exclude_defaults` | If `True`, exclude fields that have their default values              |
| `exclude_none`     | If `True`, exclude fields whose value is `None`                       |

In [15]:
from pydantic import BaseModel
from typing import Optional

class Address(BaseModel):
    city: str
    state: str
    pin: int 

class Patient(BaseModel):
    name: str
    gender: str
    age: int 
    address: Address
    phone: Optional[str] = None   # optional field

# Sample data
patient = Patient(
    name="Ankush",
    gender="Male",
    age=26,
    address=Address(city="Jamner", state="Maharashtra", pin=424206),
    phone=None
)

In [16]:
print(patient.model_dump()) # full serilization 

{'name': 'Ankush', 'gender': 'Male', 'age': 26, 'address': {'city': 'Jamner', 'state': 'Maharashtra', 'pin': 424206}, 'phone': None}


In [17]:
print(patient.model_dump(include={"name", "age"})) 

{'name': 'Ankush', 'age': 26}


In [18]:
print(patient.model_dump(include={"name", "age"}))

{'name': 'Ankush', 'age': 26}


In [19]:
print(patient.model_dump(exclude={"address", "phone"}))

{'name': 'Ankush', 'gender': 'Male', 'age': 26}


In [20]:
patient_partial = Patient(
    name="Ankush",
    gender="Male",
    age=26,
    address=Address(city="Jamner", state="Maharashtra", pin=424206)
)

print(patient_partial.model_dump(exclude_unset=True))  

{'name': 'Ankush', 'gender': 'Male', 'age': 26, 'address': {'city': 'Jamner', 'state': 'Maharashtra', 'pin': 424206}}


In [21]:
print(patient.model_dump_json())

{"name":"Ankush","gender":"Male","age":26,"address":{"city":"Jamner","state":"Maharashtra","pin":424206},"phone":null}
